# <a id='toc1_'></a>[Better model evaluation](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Better model evaluation](#toc1_)    
  - [Cross Validation](#toc1_1_)    
    - [What is cross-validation?](#toc1_1_1_)    
      - [Let's talk about bears](#toc1_1_1_1_)    
    - [Why do we do cross-validation?](#toc1_1_2_)    
    - [**Why cross-validation?**](#toc1_1_3_)    
  - [Types of cross-validation (CV)](#toc1_2_)    
    - [Stratified K-Fold Cross Validation](#toc1_2_1_)    
  - [Repeated KFold](#toc1_3_)    
    - [**How to choose K?**](#toc1_3_1_)    
    - [Shuffle Split](#toc1_3_2_)    
  - [Stratified Shuffle Split](#toc1_4_)    
  - [Time Series Cross Validation](#toc1_5_)    
  - [Extra: Leave-One-Out Cross-Validation   ](#toc1_6_)    
- [Pickling](#toc2_)    
  - [Save the model](#toc2_1_)    
  - [Load the model](#toc2_2_)    
  - [Save the data](#toc2_3_)    
  - [Load the data](#toc2_4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [12]:
from sklearn.datasets import  fetch_california_housing, load_breast_cancer
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_validate, StratifiedShuffleSplit, ShuffleSplit, KFold, RepeatedKFold, GroupKFold, StratifiedKFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score

## <a id='toc1_1_'></a>[Cross Validation](#toc0_)

### <a id='toc1_1_1_'></a>[What is cross-validation?](#toc0_)

**A way to get more accurate evaluation metrics**. 

Cross-validation allows you to test your model on multiple test sets, which gives us a better idea of how the model will work in the wild. However, it only works if the data and the data splits are representative of the real world data!


#### <a id='toc1_1_1_1_'></a>[Let's talk about bears](#toc0_)
If you were to build a bear classifier that would help tourists in your reservation to avoid bears and used only pictures from the internet, you may not be able to help. 

Why?  


Because bear pictures on the internet usually look like this:

![](../../../img/bear_image.png)

**A good quality, close-up picture of a bear**. But if you were to set up a surveillance camera to monitor bears in the wild you'd see something more like this:

![](../../../img/bear_wild.png)

**A poor quality, far away picture of a bear, with cubs too this time.**

In this scenario, our model is great on the training data (internet images), but it would perform poorly on the test data (surveillance camera images)

Assuming our data is representative of the population, cross-validation can also help us find the optimal hyperparameters for our model. More on that in Hyperparameter Tuning class.

### <a id='toc1_1_2_'></a>[Why do we do cross-validation?](#toc0_)

**Because we care about generalization.**

If we spent 3 months building a model to get our business to generate more revenue, e.g. optimizing flight prices, we would want our model to generalize well. Although it's useful in business (and widely used, especially with small amounts of data), it's even more useful in high-stakes settings, e.g. hospitals, police. 

This is because these fields generally don't have the most updated technology and will not update their models as needed once they realize the data seen by the model in real-life is very different from the data seen by the model during training (i.e. *data drift*). That's why the models used in those fields (e.g. cancer detection, perpetrator recidivism) need to be as good as they can be at the time of the building.

Most big companies do not rely on static models to predict consumer behaviour, but instead regularly track performance and update models. This is more of an MLOps topic and we won't cover it here but it's something you should be aware of. Keywords to search for: *model evaluation*, *concept drift*, *data drift*.

### How do we do cross-validation?

We split our data into multiple parts and use them for training and evaluation, one by one.

![](../../../img/crossval.png)

In [3]:
california = fetch_california_housing()
print(california["DESCR"])

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

In [4]:
df_cali = pd.DataFrame(california["data"], columns = california["feature_names"])
df_cali["median_house_value"] = california["target"]

df_cali.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,median_house_value
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [9]:
features = df_cali.drop(columns = ["median_house_value","AveOccup", "Population", "AveBedrms"])
labels = df_cali["median_house_value"]

In [10]:
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)

In [11]:
cv_splitter = KFold(n_splits=5, shuffle=False)
cross_validate(rf_model, features, labels, cv=cv_splitter)

{'fit_time': array([3.49588466, 3.56531835, 3.90509176, 3.67472696, 3.67457604]),
 'score_time': array([0.02841306, 0.02430534, 0.02716351, 0.02465272, 0.02805901]),
 'test_score': array([0.43850963, 0.58579275, 0.64412899, 0.52544421, 0.5969133 ])}

## <a id='toc1_2_'></a>[Types of cross-validation (CV)](#toc0_)

### <a id='toc1_3_2_'></a>[Shuffle Split](#toc0_)

This CV method makes sure to scramble your data a bit before the train test split! This is so you approximate a random selection of data points. You do this because you want your model to not change much when the order of the data points changes. 

The train-test split we used before already does this internally - this is why we had to set a `random_state`!

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_008.png)  
(Source: [3.1. Cross-validation: evaluating estimator performance, scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators))

### <a id='toc1_2_1_'></a>[Stratified K-Fold Cross Validation](#toc0_)

This method makes sure your target is equally represented in train-test sets. If you have imbalanced data, e.g. 30% positive 70% negative, the same %s will be present in both your train and test sets.

You should always **stratify based on the target** - which is why `sklearn` does this by default with its cross-validation function.

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_009.png)  
(Source: [3.1. Cross-validation: evaluating estimator performance, scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators))

In [13]:
cv_splitter = StratifiedKFold(n_splits=5, shuffle=False)
cross_validate(rf_model, features, labels, cv=cv_splitter)

ValueError: Supported target types are: ('binary', 'multiclass'). Got 'continuous' instead.

Why did we get this error?

### <a id='toc1_4_'></a>[Stratified Shuffle Split](#toc0_)

Same as the stratified CV we saw before, but now shuffled!

In [ ]:
# Set up the cross validator
cv_sss = StratifiedShuffleSplit(n_splits=5, test_size=0.25, random_state=0)
cv_sss.get_n_splits(features, labels)

In [ ]:
# Check what the stratified shuffle split does
for i, (train_indices, test_indices) in enumerate(cv_sss.split(features, labels)):
    print('Split no:', i)
    print('Train indices:', train_indices[:5])
    print('Test indices:', test_indices[:5])

In [ ]:
# Now see it in action! ...manually
results = []
for train_index, test_index in cv_sss.split(features, labels):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    rf_model.fit(X_train, y_train)
    pred = rf_model.predict(X_test)
    results.append(accuracy_score(y_test, pred))

In [ ]:
results

In [ ]:
# And now using the sklearn 
scores = cross_val_score(rf_model, features, labels, scoring='accuracy', cv=cv_sss, n_jobs=-1)
print(scores)
scores.mean()

### <a id='toc1_3_'></a>[Repeated KFold](#toc0_)

In [ ]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# Initialize model and cross validate with 10 folds
scores = cross_val_score(rf_model, features, labels, scoring='accuracy', cv=cv, n_jobs=-1)
print(scores)
scores.mean()

In [ ]:
# Use a different scoring metric
scores = cross_val_score(rf_model, features, labels, scoring='recall', cv=cv, n_jobs=-1)
print(scores)
scores.mean()

### <a id='toc1_3_1_'></a>[**How to choose K?**](#toc0_)

> Typical values for k are k=3, k=5, and k=10, with 10 representing the most common value. This is because, given extensive testing, 10-fold cross-validation provides a good balance of low computational cost and low bias in the estimate of model performance as compared to other k values and a single train-test split. [$^{[3]}$](https://machinelearningmastery.com/loocv-for-evaluating-machine-learning-algorithms/)

### <a id='toc1_5_'></a>[Extra: Time Series Cross Validation](#toc0_)

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_cv_indices_013.png)   
(Source: [3.1. Cross-validation: evaluating estimator performance, scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators))

In [ ]:
occupancy = pd.read_csv('https://raw.githubusercontent.com/sabinagio/data-analytics/main/data/occupancy.csv')
occupancy.set_index('date', inplace=True)
occupancy.head()

In [ ]:
features = occupancy.drop('Occupancy', axis=1)
labels = occupancy['Occupancy']

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

# Set up the cross validator
ts_sss = TimeSeriesSplit(n_splits=6)
ts_sss.get_n_splits(features)

In [ ]:
# Review how the time series split works
for i, (train_index, test_index) in enumerate(ts_sss.split(features)):
    print('Split no:', i)
    print('Train set size:', len(train_index))
    print('Test set size:', len(test_index))

In [ ]:
# And see it in action!... manually
results = []
for train_index, test_index in ts_sss.split(features):
    X_train, X_test = features.iloc[train_index], features.iloc[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    rf_model.fit(X_train, y_train)
    pred = rf_model.predict(X_test)
    results.append(accuracy_score(y_test, pred))

In [ ]:
results

In [ ]:
# And now using the sklearn 
scores = cross_val_score(rf_model, features, labels, scoring='accuracy', cv=ts_sss, n_jobs=-1)
print(scores)
scores.mean()

## <a id='toc1_6_'></a>Extra: [Leave-One-Out Cross-Validation](https://machinelearningmastery.com/loocv-for-evaluating-machine-learning-algorithms/)    [&#8593;](#toc0_)

# <a id='toc2_'></a>[Pickling](#toc0_)

We can pickle many things: ML models, pandas dataframes

In [ ]:
import pickle

## <a id='toc2_1_'></a>[Save the model](#toc0_)

In [ ]:
with open('rf_model.pkl', 'wb') as file:
    pickle.dump(rf_model, file)

In [ ]:
pickle.dump(rf_model, open('rf_model.pkl', 'wb'))

## <a id='toc2_2_'></a>[Load the model](#toc0_)

In [ ]:
with open('rf_model.pkl', 'rb') as file:
    rf_model = pickle.load(file)

In [ ]:
rf_model = pickle.load(open('rf_model.pkl', 'rb'))

## <a id='toc2_3_'></a>[Save the data](#toc0_)

In [ ]:
X_train.to_pickle('train_data.pkl')
y_train.to_pickle('train_label.pkl')

X_test.to_pickle('test_data.pkl')
y_test.to_pickle('test_label.pkl')

## <a id='toc2_4_'></a>[Load the data](#toc0_)

In [ ]:
X_train = pd.read_pickle('train_data.pkl')
X_train